In [1]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components

In [2]:
def initBonds(Lx, Ly):
    if Lx < 3 or Ly < 3:
        print("Lx,Ly size must be bigger than 2!")
        return 0
    
    def site(x,y):
        return y*Lx + x
    
    bonds = []
    for y in range(Ly):
        for x in range(Lx):
            n = site(x,y)
            m1 = site((x+1)%Lx,y)
            m2 = site(x,(y+1)%Ly)
            bonds.append([n,m1])
            bonds.append([n,m2])
            
    return np.array(bonds)

In [3]:
def initSpins(N):
    return 2*(np.random.randint(0,2,size=N)-0.5)

In [4]:
def getWeigths(spins, bonds, T):
    parallel = (spins[bonds[:, 0]] == spins[bonds[:, 1]])
    
    weights = []
    prob = np.exp(-2/T)
    for i in parallel:
        if not i:
            weights.append(0)
        elif np.random.random() < prob:
            weights.append(0)
        else:
            weights.append(1)
    
    return np.array(weights)

In [38]:
def flipClusters(Nclusters, clusters, spins):
    prob = np.random.rand(Nclusters)
    flip = prob > 0.5

    for i in range(Nclusters):
        if flip[i]:
            flipcluster = (clusters == i)
            spins[flipcluster] = -spins[flipcluster]

In [39]:
def getMagnetization(spins):
    return spins.sum()

In [40]:
def getEnergy(spins,bonds):
    return sum(spins[bonds[:, 0]]*spins[bonds[:, 1]])

In [41]:
def SwendsenWangUpdate(spins, bonds, T):
    N = len(spins)
    weights = getWeigths(spins, bonds, T)
    sparseWeights = csr_matrix((weights, (bonds[:, 0], bonds[:, 1])), shape=(N, N))
    sparseWeights += csr_matrix((weights, (bonds[:, 1], bonds[:, 0])), shape=(N, N))
    Nclusters, clusters = connected_components(sparseWeights)
    flipClusters(Nclusters, clusters, spins)
    # print(Nclusters, clusters)

In [68]:
def runMonteCarlo(spins, bonds, T, N):
    energy = []
    magnet = []
    for i in range(N):
        SwendsenWangUpdate(spins,bonds,T)
        energy.append(getEnergy(spins, bonds))
        magnet.append(getMagnetization(spins))
    return np.array(energy), np.array(magnet)

In [73]:
def main():
    Lx,Ly = 4,4 # 2Lx*Ly bonds
    N = Lx * Ly
    T = 2
    
    bonds = initBonds(Lx,Ly)
    spins = initSpins(N)
    E, M = runMonteCarlo(spins,bonds,T,10)
    print(E,"\n",M)

In [74]:
if __name__ == '__main__':
    main()

[ 4. 12. 16.  8. 32. 32. 24. 32. 32. 32.] 
 [ -8.   8.  10.   8. -16.  16.  14. -16.  16.  16.]
